In [1]:
from datasets import load_dataset
import markovify

dataset = load_dataset("biglam/gutenberg-poetry-corpus")

c:\Users\Albert\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("biglam/gutenberg-poetry-corpus")
dataset['train'] = dataset['train']

### a)

In [78]:
text = markovify.Text(dataset['train']['line'])
poemlines = []
for i in range(5):
    poemlines.append(text.make_sentence())


In [79]:
poemlines

['As in a fierie Tempest shall be silver;',
 "Essential Powers, nor by his own strength, this place may lye expos'd",
 'More lovely then PANDORA, whom the great boaster,',
 'Bent aside the Devil stood,',
 'Smoked a little longer,']

In [80]:
poem = {'line': poemlines}

### b)

In [81]:
from textblob import TextBlob
import pandas as pd

In [82]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

def getSentimentCategory(polarity):
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

def sentiment(poem):

    poem['subjectivity'] = poem['line'].apply(getSubjectivity)
    poem['polarity'] = poem['line'].apply(getPolarity)
    poem['sentiment'] = poem['polarity'].apply(getSentimentCategory)

    return poem

poem_df = pd.DataFrame(poem)


result = sentiment(poem_df)
print(result)

                                                line  subjectivity  polarity  \
0            As in a fierie Tempest shall be silver;      0.000000    0.0000   
1  Essential Powers, nor by his own strength, thi...      0.650000    0.3000   
2  More lovely then PANDORA, whom the great boaster,      0.666667    0.6000   
3                        Bent aside the Devil stood,      0.000000    0.0000   
4                            Smoked a little longer,      0.500000   -0.1875   

  sentiment  
0   neutral  
1  positive  
2  positive  
3   neutral  
4  negative  


In [83]:
poem_df

,line,subjectivity,polarity,sentiment
0,As in a fierie Tempest shall be silver;,0.000000,0.0000,neutral
1,"Essential Powers, nor by his own strength, thi...",0.650000,0.3000,positive
2,"More lovely then PANDORA, whom the great boaster,",0.666667,0.6000,positive
3,"Bent aside the Devil stood,",0.000000,0.0000,neutral
4,"Smoked a little longer,",0.500000,-0.1875,negative


### c)

In [10]:
import gensim.downloader
model = gensim.downloader.load("word2vec-google-news-300") 

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [43]:
from numpy.random import randint

def find_synonym(word):
    try:
        similar_words = model.most_similar(word, topn=10)
        for similar_word, _ in similar_words:
            if similar_word.lower() != word.lower():
                return similar_word
    except KeyError:
        return word
    return word

def replace_with_sinonims(poem):
    new_poem = []
    for line in poem:
        new_vers = ''
        for word in line.split(' '):
            p = randint(1,100)
            if p % 2 == 0:
                try:
                    word = find_synonym(word)
                except:
                    word = word
            new_vers += word + ' '
        new_poem.append(new_vers)

    return new_poem

new_poem = replace_with_sinonims(poemlines)
for line in new_poem:
    print(line)

for line in poem['line']:
    print(line)

The working of devastation, 
As stands really these, they chanted. 
Reasons well chief; amongst those this seats alongside him, 
The work of devastation,
As stood like these, they chanted.
Reason as chief; among these the seat beside him,


### BLEU Score

In [88]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

reference = [[word for word in line] for line in dataset['train']['line']]
candidate = [word for line in poemlines for word in line ]
score = sentence_bleu(reference, candidate)

print(score)

0.8919037207747021
